# import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import click
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import time
import functools
from argparse import Namespace
from pathlib import Path
!pip install scapy
from scapy.compat import raw
from scapy.layers.inet import IP, UDP
from scapy.layers.l2 import Ether
from scapy.packet import Padding
from scipy import sparse
from scapy.layers.dns import DNS
from scapy.layers.inet import TCP
from scapy.packet import Padding
from scapy.utils import rdpcap


#Preprocessing

Labeling the App and Traffic different types

In [ ]:
# for app identification
PREFIX_TO_APP_ID = {
    # AIM chat
    'aim_chat_3a': 0,
    'aim_chat_3b': 0,
    'aimchat1': 0,
    'aimchat2': 0,
    # Email
    'email1a': 1,
    'email1b': 1,
    'email2a': 1,
    'email2b': 1,
    # Facebook
    'facebook_audio1a': 2,
    'facebook_audio1b': 2,
    'facebook_audio2a': 2,
    'facebook_audio2b': 2,
    'facebook_audio3': 2,
    'facebook_audio4': 2,
    'facebook_chat_4a': 2,
    'facebook_chat_4b': 2,
    'facebook_video1a': 2,
    'facebook_video1b': 2,
    'facebook_video2a': 2,
    'facebook_video2b': 2,
    'facebookchat1': 2,
    'facebookchat2': 2,
    'facebookchat3': 2,
    # FTPS
    'ftps_down_1a': 3,
    'ftps_down_1b': 3,
    'ftps_up_2a': 3,
    'ftps_up_2b': 3,
    # Gmail
    'gmailchat1': 4,
    'gmailchat2': 4,
    'gmailchat3': 4,
    # Hangouts
    'hangout_chat_4b': 5,
    'hangouts_audio1a': 5,
    'hangouts_audio1b': 5,
    'hangouts_audio2a': 5,
    'hangouts_audio2b': 5,
    'hangouts_audio3': 5,
    'hangouts_audio4': 5,
    'hangouts_chat_4a': 5,
    'hangouts_video1b': 5,
    'hangouts_video2a': 5,
    'hangouts_video2b': 5,
    # ICQ
    'icq_chat_3a': 6,
    'icq_chat_3b': 6,
    'icqchat1': 6,
    'icqchat2': 6,
    # Netflix
    'netflix1': 7,
    'netflix2': 7,
    'netflix3': 7,
    'netflix4': 7,
    # SCP
    'scp1': 8,
    'scpdown1': 8,
    'scpdown2': 8,
    'scpdown3': 8,
    'scpdown4': 8,
    'scpdown5': 8,
    'scpdown6': 8,
    'scpup1': 8,
    'scpup2': 8,
    'scpup3': 8,
    'scpup5': 8,
    'scpup6': 8,
    # SFTP
    'sftp1': 9,
    'sftp_down_3a': 9,
    'sftp_down_3b': 9,
    'sftp_up_2a': 9,
    'sftp_up_2b': 9,
    'sftpdown1': 9,
    'sftpdown2': 9,
    'sftpup1': 9,
    # Skype
    'skype_audio1a': 10,
    'skype_audio1b': 10,
    'skype_audio2a': 10,
    'skype_audio2b': 10,
    'skype_audio3': 10,
    'skype_audio4': 10,
    'skype_chat1a': 10,
    'skype_chat1b': 10,
    'skype_file1': 10,
    'skype_file2': 10,
    'skype_file3': 10,
    'skype_file4': 10,
    'skype_file5': 10,
    'skype_file6': 10,
    'skype_file7': 10,
    'skype_file8': 10,
    'skype_video1a': 10,
    'skype_video1b': 10,
    'skype_video2a': 10,
    'skype_video2b': 10,
    # Spotify
    'spotify1': 11,
    'spotify2': 11,
    'spotify3': 11,
    'spotify4': 11,
    # Torrent
    'torrent01': 12,
    # Tor
    'torfacebook': 13,
    'torgoogle': 13,
    'tortwitter': 13,
    'torvimeo1': 13,
    'torvimeo2': 13,
    'torvimeo3': 13,
    'toryoutube1': 13,
    'toryoutube2': 13,
    'toryoutube3': 13,
    # Vimeo
    'vimeo1': 14,
    'vimeo2': 14,
    'vimeo3': 14,
    'vimeo4': 14,
    # Voipbuster
    'voipbuster1b': 15,
    'voipbuster2b': 15,
    'voipbuster3b': 15,
    'voipbuster_4a': 15,
    'voipbuster_4b': 15,
    # Youtube
    'youtube1': 16,
    'youtube2': 16,
    'youtube3': 16,
    'youtube4': 16,
    'youtube5': 16,
    'youtube6': 16,
    'youtubehtml5_1': 16,
}

ID_TO_APP = {
    0: 'AIM Chat',
    1: 'Email',
    2: 'Facebook',
    3: 'FTPS',
    4: 'Gmail',
    5: 'Hangouts',
    6: 'ICQ',
    7: 'Netflix',
    8: 'SCP',
    9: 'SFTP',
    10: 'Skype',
    11: 'Spotify',
    12: 'Torrent',
    13: 'Tor',
    14: 'Vimeo',
    15: 'Voipbuster',
    16: 'Youtube',
}

# for traffic identification
PREFIX_TO_TRAFFIC_ID = {
    # Chat
    'aim_chat_3a': 0,
    'aim_chat_3b': 0,
    'aimchat1': 0,
    'aimchat2': 0,
    'facebook_chat_4a': 0,
    'facebook_chat_4b': 0,
    'facebookchat1': 0,
    'facebookchat2': 0,
    'facebookchat3': 0,
    'hangout_chat_4b': 0,
    'hangouts_chat_4a': 0,
    'icq_chat_3a': 0,
    'icq_chat_3b': 0,
    'icqchat1': 0,
    'icqchat2': 0,
    'skype_chat1a': 0,
    'skype_chat1b': 0,
    # Email
    'email1a': 1,
    'email1b': 1,
    'email2a': 1,
    'email2b': 1,
    # File Transfer
    'ftps_down_1a': 2,
    'ftps_down_1b': 2,
    'ftps_up_2a': 2,
    'ftps_up_2b': 2,
    'sftp1': 2,
    'sftp_down_3a': 2,
    'sftp_down_3b': 2,
    'sftp_up_2a': 2,
    'sftp_up_2b': 2,
    'sftpdown1': 2,
    'sftpdown2': 2,
    'sftpup1': 2,
    'skype_file1': 2,
    'skype_file2': 2,
    'skype_file3': 2,
    'skype_file4': 2,
    'skype_file5': 2,
    'skype_file6': 2,
    'skype_file7': 2,
    'skype_file8': 2,
    # Streaming
    'vimeo1': 3,
    'vimeo2': 3,
    'vimeo3': 3,
    'vimeo4': 3,
    'youtube1': 3,
    'youtube2': 3,
    'youtube3': 3,
    'youtube4': 3,
    'youtube5': 3,
    'youtube6': 3,
    'youtubehtml5_1': 3,
    # Torrent
    'torrent01': 4,
    # VoIP
    'facebook_audio1a': 5,
    'facebook_audio1b': 5,
    'facebook_audio2a': 5,
    'facebook_audio2b': 5,
    'facebook_audio3': 5,
    'facebook_audio4': 5,
    'hangouts_audio1a': 5,
    'hangouts_audio1b': 5,
    'hangouts_audio2a': 5,
    'hangouts_audio2b': 5,
    'hangouts_audio3': 5,
    'hangouts_audio4': 5,
    'skype_audio1a': 5,
    'skype_audio1b': 5,
    'skype_audio2a': 5,
    'skype_audio2b': 5,
    'skype_audio3': 5,
    'skype_audio4': 5,
    # VPN: Chat
    'vpn_aim_chat1a': 6,
    'vpn_aim_chat1b': 6,
    'vpn_facebook_chat1a': 6,
    'vpn_facebook_chat1b': 6,
    'vpn_hangouts_chat1a': 6,
    'vpn_hangouts_chat1b': 6,
    'vpn_icq_chat1a': 6,
    'vpn_icq_chat1b': 6,
    'vpn_skype_chat1a': 6,
    'vpn_skype_chat1b': 6,
    # VPN: File Transfer
    'vpn_ftps_a': 7,
    'vpn_ftps_b': 7,
    'vpn_sftp_a': 7,
    'vpn_sftp_b': 7,
    'vpn_skype_files1a': 7,
    'vpn_skype_files1b': 7,
    # VPN: Email
    'vpn_email2a': 8,
    'vpn_email2b': 8,
    # VPN: Streaming
    'vpn_vimeo_a': 9,
    'vpn_vimeo_b': 9,
    'vpn_youtube_a': 9,
    # VPN: Torrent
    'vpn_bittorrent': 10,
    # VPN VoIP
    'vpn_facebook_audio2': 11,
    'vpn_hangouts_audio1': 11,
    'vpn_hangouts_audio2': 11,
    'vpn_skype_audio1': 11,
    'vpn_skype_audio2': 11,
}

ID_TO_TRAFFIC = {
    0: 'Chat',
    1: 'Email',
    2: 'File Transfer',
    3: 'Streaming',
    4: 'Torrent',
    5: 'Voip',
    6: 'VPN: Chat',
    7: 'VPN: File Transfer',
    8: 'VPN: Email',
    9: 'VPN: Streaming',
    10: 'VPN: Torrent',
    11: 'VPN: Voip',
}


#Packet Preprocessing

In [ ]:
def read_pcap(path: Path):
    packets = rdpcap(str(path))
    return packets


def should_omit_packet(packet):
    # SYN, ACK or FIN flags set to 1 and no payload
    if TCP in packet and (packet.flags & 0x13):
        # not payload or contains only padding
        layers = packet[TCP].payload.layers()
        if not layers or (Padding in layers and len(layers) == 1):
            return True

    # DNS segment
    if DNS in packet:
        return True

    return False


def remove_ether_header(packet):
    if Ether in packet:
        return packet[Ether].payload
    return packet


def mask_ip(packet):
    if IP in packet:
        packet[IP].src = '0.0.0.0'
        packet[IP].dst = '0.0.0.0'

    return packet


def pad_udp(packet):
    if UDP in packet:
        # get layers after udp
        layer_after = packet[UDP].payload.copy()

        # build a padding layer
        pad = Padding()
        pad.load = '\x00' * 12

        layer_before = packet.copy()
        layer_before[UDP].remove_payload()
        packet = layer_before / pad / layer_after

        return packet

    return packet


def packet_to_sparse_array(packet, max_length=1500):
    # Original implementation
    print('raw packet\n', raw(packet))
    arr = np.frombuffer(raw(packet), dtype=np.uint8)[0: max_length] / 255  # divide each byte by 255
    print('arr before padding\n', arr)
    if len(arr) < max_length:
        pad_width = max_length - len(arr)
        arr = np.pad(arr, pad_width=(0, pad_width), constant_values=0)  # padding the packet
        print('arr after padding\n', arr)

    arr = sparse.csr_matrix(arr)
    return arr

In [ ]:
def transform_packet(packet):
    if should_omit_packet(packet):
        return None
    print('packet before delete\n', packet)
    packet = remove_ether_header(packet)
    packet = pad_udp(packet)
    packet = mask_ip(packet)
    print('packet after delete\n', packet)

    arr = packet_to_sparse_array(packet) # if original implementation (original packet length)

    return arr


def transform_pcap(path, output_path: Path = None, output_batch_size=10000):
    if Path(str(output_path.absolute()) + '_SUCCESS').exists():
        print(output_path, 'Done')
        return

    print('Processing', path)

    rows = []
    batch_index = 0
    for i, packet in enumerate(read_pcap(path)):
        arr = transform_packet(packet)
        if arr is not None:
            # get labels for app identification
            # print(arr)
            prefix = path.name.split('.')[0].lower()
            app_label = PREFIX_TO_APP_ID.get(prefix)
            traffic_label = PREFIX_TO_TRAFFIC_ID.get(prefix)
            row = {
                'app_label': app_label,
                'traffic_label': traffic_label,
                'feature': arr.todense().tolist()[0]
            }
            # print(row)
            rows.append(row)

        # write every batch_size packets, by default 10000
        if rows and i > 0 and i % output_batch_size == 0:
            part_output_path = Path(str(output_path.absolute()) + f'_part_{batch_index:04d}.csv')
            df = pd.DataFrame(rows)
            df.to_csv(part_output_path)
            batch_index += 1
            rows.clear()

    # final write
    if rows:
        df = pd.DataFrame(rows)
        part_output_path = Path(str(output_path.absolute()) + f'_part_{batch_index:04d}.csv')
        df.to_csv(part_output_path)

    # write success file
    with Path(str(output_path.absolute()) + '_SUCCESS').open('w') as f:
        f.write('')

    print(output_path, 'Done')




In [ ]:
@click.command()
@click.option('-s', '--source', help='path to the directory containing raw pcap files', required=True)
@click.option('-t', '--target', help='path to the directory for persisting preprocessed files', required=True)
def main(source, target):
    data_dir_path = Path(source)
    target_dir_path = Path(target)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    for pcap_path in sorted(data_dir_path.iterdir()):
        transform_pcap(pcap_path, target_dir_path / (pcap_path.name + '.transformed'))

if __name__ == '__main__':
    main()

#Load data after preprocessing

In [ ]:
def preprocessing_features(df_train):
    first_drop = pd.DataFrame(df_train['feature'].str.split('['))
    x = pd.DataFrame(first_drop.feature.values.tolist()).add_prefix('feature_')
    x.drop('feature_0', axis=1, inplace=True)
    second_drop = pd.DataFrame(x['feature_1'].str.split(']'))
    x = pd.DataFrame(second_drop.feature_1.values.tolist()).add_prefix('feature_')
    x.drop('feature_1', axis=1, inplace=True)
    x = x.rename({'feature_0':'feature'}, axis=1)
    features = pd.DataFrame(x['feature'].str.split(','))
    x = pd.DataFrame(features.feature.values.tolist()).add_prefix('feature_')
    x = x.fillna(0)
    x = x.apply(np.float64)
    return x

In [ ]:
from torch.utils.data import Dataset

class PcapDataset(Dataset):
    def __init__(self, df_train):
      x = preprocessing_features(df_train)
      y = df_train['traffic_label']
      #y = df_train['app_label']
      y = y.apply(np.float64)

      self.x_train = torch.tensor(x.values, dtype=torch.float64)
      self.y_train = torch.tensor(y.values, dtype=torch.float64)

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self, idx):
        return self.x_train[idx], self.y_train[idx]

In [ ]:
# get the data from csv

csvfile = pd.read_csv('/content/drive/MyDrive/combine.csv')
df = csvfile.sample(20000)

df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.8

train = df[msk]
test = df[~msk]
train.drop(columns=['split'], inplace=True)
test.drop(columns=['split'], inplace=True)

train.dropna(axis=0, how='any', inplace=True)
test.dropna(axis=0, how='any', inplace=True)

pcap_train = PcapDataset(train)
train_loader = torch.utils.data.DataLoader(pcap_train, batch_size=25, shuffle=True, drop_last=True)
pcap_test = PcapDataset(test)
test_loader = torch.utils.data.DataLoader(pcap_test, batch_size=25, shuffle=False, drop_last=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


#CNN model

In [ ]:
class CNN(nn.Module):

  def __init__(self,n_classes):
      super(CNN, self).__init__()
      self.conv1 = nn.Conv1d(1, 200, 5, 2, 0)
      self.conv2 = nn.Conv1d(200, 100, 4, 1, 0)

      self.pool = nn.AvgPool1d(2)
      self.dropout = nn.Dropout(p=0.25)

      self.fc1 = nn.Linear(18500, 200)
      self.fc2 = nn.Linear(200, 100)
      self.fc3 = nn.Linear(100, 50)
      self.out = nn.Linear(50,n_classes)

  def forward(self, x):
      x = F.avg_pool1d(F.relu(self.conv1(x)),2)
      x = self.dropout(x)
      x = F.avg_pool1d(F.relu(self.conv2(x)),2)
      x = self.dropout(x)

      x = torch.flatten(x, start_dim=1)
      x = F.relu(self.fc1(x))
      x = self.dropout(x)     
      x = F.relu(self.fc2(x))
      x = self.dropout(x)
      x = F.relu(self.fc3(x))
      x = self.dropout(x)

      x = self.out(x)

      return x

In [ ]:
n_classes = 12 #traffic
#n_classes = 17 #App
model = CNN(n_classes).cuda()
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

CNN(
  (conv1): Conv1d(1, 200, kernel_size=(5,), stride=(2,))
  (conv2): Conv1d(200, 100, kernel_size=(4,), stride=(1,))
  (pool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=18500, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=50, bias=True)
  (out): Linear(in_features=50, out_features=12, bias=True)
)


#Auto-encoder model 

In [ ]:
class StackedAutoEncoder(nn.Module):
    def __init__(self, n_classes):
        super(StackedAutoEncoder, self).__init__()
    
        # encoder
        self.enc1 = nn.Linear(in_features=1500 , out_features=300)
        self.enc2 = nn.Linear(in_features=300, out_features=60)

        # decoder 
        self.dec1 = nn.Linear(in_features=60, out_features=300)
        self.dec2 = nn.Linear(in_features=300, out_features=1500)

        #out
        self.fc_out = nn.Linear(1500, n_classes)
        self.lsm = nn.Softmax(dim=0)


    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
  
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))

        x = self.fc_out(x)
        
        return x
        

In [ ]:
n_classes = 12 #traffic
#n_classes = 17 #App
model = StackedAutoEncoder(n_classes).cuda()
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

StackedAutoEncoder(
  (enc1): Linear(in_features=1500, out_features=300, bias=True)
  (enc2): Linear(in_features=300, out_features=60, bias=True)
  (dec1): Linear(in_features=60, out_features=300, bias=True)
  (dec2): Linear(in_features=300, out_features=1500, bias=True)
  (fc_out): Linear(in_features=1500, out_features=12, bias=True)
  (lsm): Softmax(dim=0)
)


#training CNN model

In [ ]:
# Train the model
train_loss = []
test_loss = []
interval_tuples = []
start = time.time()

for epoch in range(5):
    running_train_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        labels = labels.long()
        inputs = np.expand_dims(inputs, axis=1)
        inputs = torch.Tensor(inputs)
        inputs = inputs.cuda() # -- for GPU
        labels = labels.cuda() # -- for GPU
        
        # zero the parameters gradients
        optimizer.zero_grad()

        # forward + backward + optimization
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()
        if (i+1) % 100 == 0:
            interval_tuples.append(str((epoch + 1, i + 1)))
            print("[{}, {}] loss: {}".format(epoch + 1, i + 1, running_train_loss / 100))
            train_loss.append(running_train_loss / 100)
            running_train_loss = 0.0

stop = time.time()
print("Training time: {}".format(stop-start))

[1, 100] loss: 1.590349280834198
[1, 200] loss: 1.237583685517311
[2, 100] loss: 1.034083986878395
[2, 200] loss: 0.8310993477702141
[3, 100] loss: 0.5211205231398344
[3, 200] loss: 0.49101036347448823
[4, 100] loss: 0.36555207557976244
[4, 200] loss: 0.33150065086781977
[5, 100] loss: 0.2747922194376588
[5, 200] loss: 0.2598778015188873
Training time: 374.77481293678284


#training autoencoder

In [ ]:
# Train the model
train_loss = []
test_loss = []
interval_tuples = []
start = time.time()

for epoch in range(5):
    running_train_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        labels = labels.long()
        inputs = inputs.float()
        inputs = torch.Tensor(inputs)
        inputs = inputs.cuda() # -- for GPU
        labels = labels.cuda() # -- for GPU
        
        # zero the parameters gradients
        optimizer.zero_grad()

        outputs = model(inputs)

        outputs
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics 
        running_train_loss += loss.item()
        if (i+1) % 100 == 0:
            interval_tuples.append(str((epoch + 1, i + 1)))
            print("[{}, {}] loss: {}".format(epoch + 1, i + 1, running_train_loss / 100))
            train_loss.append(running_train_loss / 100)
            running_train_loss = 0.0

stop = time.time()
print("Training time: {}".format(stop-start))


[1, 100] loss: 1.1530652749538421
[1, 200] loss: 0.6814604786038398
[2, 100] loss: 0.5073436924815178
[2, 200] loss: 0.47935782119631765
[3, 100] loss: 0.2891769541427493
[3, 200] loss: 0.26466762710362673
[4, 100] loss: 0.1631641717813909
[4, 200] loss: 0.1942844560649246
[5, 100] loss: 0.1297033405629918
[5, 200] loss: 0.11124686865136027
Training time: 24.672285795211792


#CNN Accuracy

In [ ]:
total = 0
correct = 0
with torch.no_grad(): 
    for data in test_loader:
        inputs, labels = data
        labels = labels.float()
        inputs = np.expand_dims(inputs, axis=1)
        inputs = torch.Tensor(inputs)
        inputs = inputs.cuda() # -- for GPU
        labels = labels.cuda() # -- for GPU

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy: {}%'.format((100 * correct / total)))

Accuracy: 92.52173913043478%


#Autoencoder Accuracy

In [ ]:
total = 0
correct = 0
with torch.no_grad(): 
    for data in test_loader:
        inputs, labels = data
        labels = labels.float()
        #inputs = np.expand_dims(inputs, axis=1)
        inputs = inputs.float()
        inputs = torch.Tensor(inputs)
        inputs = inputs.cuda() # -- for GPU
        labels = labels.cuda() # -- for GPU

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy: {}%'.format((100 * correct / total)))

Accuracy: 93.97101449275362%
